# AI古籍校勘实验：用大语言模型探索文本异文

**基于大语言模型的古籍版本比对与智能校勘**

---

## 课程简介

本实验将带你体验如何使用人工智能（大语言模型）分析古籍文本差异并进行智能校勘。你将学习：

1. **古籍版本学**的基本概念与异文成因
2. **文本比对技术**：自动检测版本差异
3. **AI校勘分析**：让大模型判断哪个版本更接近原文
4. **交互式实验**：输入自己的文本，验证AI的分析能力
5. **批判性思考**：理解AI在人文研究中的价值与局限

---

## 什么是古籍校勘？

| 概念 | 说明 |
|------|------|
| 校勘 | 比较同一书籍的不同版本，找出文字差异，判断正误 |
| 异文 | 同一文献在不同版本中出现的文字差异 |
| 讹误 | 传抄或刻印过程中产生的错误 |
| 通假字 | 古代用音同或音近的字代替本字的现象 |

校勘学是研究古籍的重要基础，AI技术为这一传统学科带来了新的可能性。

---

## 技术说明

本实验使用 **硅基流动 (SiliconFlow)** 提供的大模型API服务，调用 Qwen3-8B 免费模型进行古文分析。

---

## 目录

- [第零部分: 环境配置与准备](#part0)
- [第一部分: 古籍版本差异概述](#part1)
- [第二部分: 文本比对与差异检测](#part2)
- [第三部分: AI校勘分析](#part3)
- [第四部分: 交互式探索实验](#part4)
- [第五部分: 批判性思考与讨论](#part5)
- [附录: 预置案例库与拓展资源](#appendix)

---
<a id='part0'></a>
# 第零部分: 环境配置与准备

本部分将完成所有必要的准备工作：
- 安装必要的Python库
- 配置大模型API
- 定义工具函数

**操作说明**：点击每个代码单元格左侧的「运行」按钮，或按 `Shift + Enter` 执行。

In [3]:
#@title ## 0.1 安装必要的库 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，安装本实验需要的Python库。
#@markdown
#@markdown 将安装以下库：
#@markdown - `openai`: 用于调用大模型API
#@markdown - `ipywidgets`: 交互式控件
#@markdown
#@markdown ---

print(" 正在安装依赖...")
print("-" * 50)

import subprocess
import sys

packages = [
    ('openai', 'openai'),
    ('ipywidgets', 'ipywidgets'),
]

for i, (pkg_name, import_name) in enumerate(packages, 1):
    print(f"[{i}/{len(packages)}] 检查 {pkg_name}...")
    try:
        __import__(import_name)
        print(f"    已安装")
    except ImportError:
        print(f"    正在安装...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg_name])
        print(f"    完成")

# 启用widgets
try:
    from google.colab import output
    output.enable_custom_widget_manager()
    print("\n 交互式控件已启用")
except:
    pass

print("-" * 50)
print(" 所有依赖安装完成！")

 正在安装依赖...
--------------------------------------------------
[1/2] 检查 openai...
    已安装
[2/2] 检查 ipywidgets...
    已安装

 交互式控件已启用
--------------------------------------------------
 所有依赖安装完成！


In [4]:
#@title ## 0.2 配置API密钥 { display-mode: "form" }
#@markdown ---
#@markdown ### 当前配置：硅基流动 (SiliconFlow)
#@markdown
#@markdown 本实验默认使用硅基流动的免费API服务，Qwen3-8B模型完全免费。
#@markdown
#@markdown **获取密钥**：https://cloud.siliconflow.cn -> 注册登录 -> API密钥
#@markdown
#@markdown ---
#@markdown ### 其他可选平台
#@markdown
#@markdown | 平台 | BASE_URL | 模型 |
#@markdown |------|----------|----------|
#@markdown | 硅基流动 | https://api.siliconflow.cn/v1 | Qwen/Qwen3-8B（免费） |
#@markdown | DeepSeek | https://api.deepseek.com | deepseek-chat (需充值) |
#@markdown | 通义千问 | https://dashscope.aliyuncs.com/compatible-mode/v1 | qwen-turbo |
#@markdown | OpenAI | https://api.openai.com/v1 | 无免费 |
#@markdown
#@markdown 如需切换平台，修改下方三个参数即可。
#@markdown
#@markdown ---

# ========== 配置区（可自由修改）==========
API_KEY = "sk-navdthzhkiaxcaodurrhjiyljntfvanmggawyocwavicplkd"  #@param {type:"string"}
BASE_URL = "https://api.siliconflow.cn/v1"  #@param {type:"string"}
MODEL = "Qwen/Qwen3-8B"  #@param {type:"string"}
# ==========================================

print("=" * 50)
print(" API配置")
print("=" * 50)

from openai import OpenAI

client = OpenAI(
    api_key=API_KEY,
    base_url=BASE_URL
)

# 测试连接
print(f"\n 平台地址: {BASE_URL}")
print(f" 使用模型: {MODEL}")

try:
    test_response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": "你好"}],
        max_tokens=10
    )
    print(" 连接状态: 成功")
    print("\n 可以开始实验")
except Exception as e:
    print(f" 连接状态: 失败")
    print(f" 错误信息: {e}")
    print("\n 请检查API密钥和平台地址是否正确")

print("=" * 50)

 API配置

 平台地址: https://api.siliconflow.cn/v1
 使用模型: Qwen/Qwen3-8B
 连接状态: 成功

 可以开始实验


In [5]:
#@title ## 0.3 定义工具函数 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，定义文本比对和AI分析所需的函数。
#@markdown ---

import difflib
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

def find_differences(text1, text2):
    """
    找出两个文本的逐字差异
    返回: 差异列表，每个元素为 (位置, 版本A字符, 版本B字符)
    """
    differences = []
    matcher = difflib.SequenceMatcher(None, text1, text2)

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'replace':
            differences.append({
                'type': '替换',
                'position': i1,
                'text_a': text1[i1:i2],
                'text_b': text2[j1:j2]
            })
        elif tag == 'delete':
            differences.append({
                'type': '删除',
                'position': i1,
                'text_a': text1[i1:i2],
                'text_b': '(无)'
            })
        elif tag == 'insert':
            differences.append({
                'type': '插入',
                'position': i1,
                'text_a': '(无)',
                'text_b': text2[j1:j2]
            })

    return differences


def highlight_differences(text1, text2):
    """
    生成带高亮标记的文本
    差异部分用【】标出
    """
    result1, result2 = [], []
    matcher = difflib.SequenceMatcher(None, text1, text2)

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'equal':
            result1.append(text1[i1:i2])
            result2.append(text2[j1:j2])
        elif tag == 'replace':
            result1.append(f"【{text1[i1:i2]}】")
            result2.append(f"【{text2[j1:j2]}】")
        elif tag == 'delete':
            result1.append(f"【{text1[i1:i2]}】")
        elif tag == 'insert':
            result2.append(f"【{text2[j1:j2]}】")

    return ''.join(result1), ''.join(result2)


def ai_analyze(text1, text2, differences):
    """
    调用AI分析两个版本的差异
    """
    diff_desc = "\n".join([f"- 位置{d['position']}: 版本A为「{d['text_a']}」，版本B为「{d['text_b']}」({d['type']})"
                          for d in differences])

    prompt = f"""你是一位古籍校勘专家。请分析以下两个古文版本的差异：

版本A：{text1}
版本B：{text2}

已检测到的差异：
{diff_desc}

请从以下角度进行分析：
1. 逐一分析每处差异的性质（通假字、形近字讹误、衍文、脱文等）
2. 判断哪个版本更可能接近原文，并说明理由
3. 如有必要，引用相关文献或版本学证据

请用专业但易懂的语言回答。"""

    response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}],
    max_tokens=2000
)

    return response.choices[0].message.content


def print_diff_table(differences):
    """
    打印差异对比表
    """
    print("\n" + "-" * 50)
    print(" 差异对比表")
    print("-" * 50)
    print(f"{'序号':<6}{'类型':<8}{'版本A':<15}{'版本B':<15}")
    print("-" * 50)
    for i, d in enumerate(differences, 1):
        print(f"{i:<6}{d['type']:<8}{d['text_a']:<15}{d['text_b']:<15}")
    print("-" * 50)


print("=" * 50)
print(" 工具函数定义完成")
print("=" * 50)
print("\n 已定义以下函数：")
print(" - find_differences(): 检测文本差异")
print(" - highlight_differences(): 生成高亮文本")
print(" - ai_analyze(): AI智能分析")
print(" - print_diff_table(): 打印差异表格")
print("\n" + "=" * 50)

 工具函数定义完成

 已定义以下函数：
 - find_differences(): 检测文本差异
 - highlight_differences(): 生成高亮文本
 - ai_analyze(): AI智能分析
 - print_diff_table(): 打印差异表格



---
<a id='part1'></a>
# 第一部分: 古籍版本差异概述

在开始实验之前，我们先了解古籍版本差异的常见类型和成因。

In [6]:
#@title ## 1.1 异文的常见类型 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，了解古籍异文的常见类型。
#@markdown ---

print("=" * 60)
print(" 古籍异文的常见类型")
print("=" * 60)

diff_types = [
    ("通假字", "用音同或音近的字代替本字", "说/悦、反/返、知/智"),
    ("形近字讹误", "因字形相似而误写", "己/已/巳、戊/戌/戍"),
    ("衍文", "多出的文字（传抄时误增）", "原文无，某版本多出字词"),
    ("脱文", "缺失的文字（传抄时遗漏）", "原文有，某版本缺失字词"),
    ("倒文", "字词顺序颠倒", "君子/子君"),
    ("异体字", "同一字的不同写法", "峰/峯、群/羣"),
]

for dtype, desc, example in diff_types:
    print(f"\n【{dtype}】")
    print(f"  说明: {desc}")
    print(f"  示例: {example}")

print("\n" + "-" * 60)
print(" 理解这些类型有助于我们分析AI的校勘建议是否合理。")
print("=" * 60)

 古籍异文的常见类型

【通假字】
  说明: 用音同或音近的字代替本字
  示例: 说/悦、反/返、知/智

【形近字讹误】
  说明: 因字形相似而误写
  示例: 己/已/巳、戊/戌/戍

【衍文】
  说明: 多出的文字（传抄时误增）
  示例: 原文无，某版本多出字词

【脱文】
  说明: 缺失的文字（传抄时遗漏）
  示例: 原文有，某版本缺失字词

【倒文】
  说明: 字词顺序颠倒
  示例: 君子/子君

【异体字】
  说明: 同一字的不同写法
  示例: 峰/峯、群/羣

------------------------------------------------------------
 理解这些类型有助于我们分析AI的校勘建议是否合理。


---
<a id='part2'></a>
# 第二部分: 文本比对与差异检测

本部分将使用Python的difflib库自动检测两个文本版本之间的差异。

In [7]:
#@title ## 2.1 案例一：《论语》"说/悦"之辨 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，分析《论语·学而》开篇的版本差异。
#@markdown
#@markdown 这是最经典的古籍异文案例之一。
#@markdown ---

print("=" * 60)
print(" 案例一：《论语·学而》版本比对")
print("=" * 60)

# 两个版本
text_a = "子曰：学而时习之，不亦说乎？有朋自远方来，不亦乐乎？"
text_b = "孔子曰：学而时习之，不亦悦乎？有朋自远方来，不亦乐乎？"

print("\n 原始文本：")
print(f" 版本A: {text_a}")
print(f" 版本B: {text_b}")

# 检测差异
differences = find_differences(text_a, text_b)

# 高亮显示
highlighted_a, highlighted_b = highlight_differences(text_a, text_b)
print("\n 差异高亮（【】内为差异部分）：")
print(f" 版本A: {highlighted_a}")
print(f" 版本B: {highlighted_b}")

# 打印差异表
print_diff_table(differences)

print(f"\n 共检测到 {len(differences)} 处差异")
print("=" * 60)

 案例一：《论语·学而》版本比对

 原始文本：
 版本A: 子曰：学而时习之，不亦说乎？有朋自远方来，不亦乐乎？
 版本B: 孔子曰：学而时习之，不亦悦乎？有朋自远方来，不亦乐乎？

 差异高亮（【】内为差异部分）：
 版本A: 子曰：学而时习之，不亦【说】乎？有朋自远方来，不亦乐乎？
 版本B: 【孔】子曰：学而时习之，不亦【悦】乎？有朋自远方来，不亦乐乎？

--------------------------------------------------
 差异对比表
--------------------------------------------------
序号    类型      版本A            版本B            
--------------------------------------------------
1     插入      (无)            孔              
2     替换      说              悦              
--------------------------------------------------

 共检测到 2 处差异


In [8]:
#@title ## 2.2 案例二：《史记》标点差异 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，分析《史记·项羽本纪》的版本差异。
#@markdown
#@markdown 这个案例展示了标点（断句）不同导致的语义差异。
#@markdown ---

print("=" * 60)
print(" 案例二：《史记·项羽本纪》版本比对")
print("=" * 60)

text_a = "项籍少时，学书不成，去；学剑，又不成。"
text_b = "项籍少时，学书不成，去学剑，又不成。"

print("\n 原始文本：")
print(f" 版本A: {text_a}")
print(f" 版本B: {text_b}")

differences = find_differences(text_a, text_b)
highlighted_a, highlighted_b = highlight_differences(text_a, text_b)

print("\n 差异高亮：")
print(f" 版本A: {highlighted_a}")
print(f" 版本B: {highlighted_b}")

print_diff_table(differences)

print("\n 思考：两种断句方式对语义有何影响？")
print(" 版本A：学书不成后离开，然后学剑也不成")
print(" 版本B：学书不成后转去学剑，又不成")
print("=" * 60)

 案例二：《史记·项羽本纪》版本比对

 原始文本：
 版本A: 项籍少时，学书不成，去；学剑，又不成。
 版本B: 项籍少时，学书不成，去学剑，又不成。

 差异高亮：
 版本A: 项籍少时，学书不成，去【；】学剑，又不成。
 版本B: 项籍少时，学书不成，去学剑，又不成。

--------------------------------------------------
 差异对比表
--------------------------------------------------
序号    类型      版本A            版本B            
--------------------------------------------------
1     删除      ；              (无)            
--------------------------------------------------

 思考：两种断句方式对语义有何影响？
 版本A：学书不成后离开，然后学剑也不成
 版本B：学书不成后转去学剑，又不成


---
<a id='part3'></a>
# 第三部分: AI校勘分析

本部分将调用大语言模型，让AI分析版本差异并给出校勘建议。

In [9]:
#@title ## 3.1 AI分析《论语》异文 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，让AI分析《论语》"说/悦"之辨。
#@markdown ---

print("=" * 60)
print(" AI校勘分析：《论语·学而》")
print("=" * 60)

text_a = "子曰：学而时习之，不亦说乎？有朋自远方来，不亦乐乎？"
text_b = "孔子曰：学而时习之，不亦悦乎？有朋自远方来，不亦乐乎？"

print("\n 正在调用AI进行分析...")
print("-" * 60)

differences = find_differences(text_a, text_b)
analysis = ai_analyze(text_a, text_b, differences)

print("\n AI分析结果：")
print("-" * 60)
print(analysis)
print("\n" + "=" * 60)

 AI校勘分析：《论语·学而》

 正在调用AI进行分析...
------------------------------------------------------------

 AI分析结果：
------------------------------------------------------------
作为古籍校勘专家，我们可以从版本学、文字学、训诂学等角度，对《论语》中两处差异进行分析。以下是对版本A与版本B的逐条校勘分析。

---

### 一、逐一分析每处差异的性质

#### **1. 位置0：「(无)」 vs 「孔」（插入）**

- **版本A**：在“子曰”前没有“孔”字。
- **版本B**：在“子曰”前插入了“孔”字，变为“孔子曰”。

**性质判断**：
- **版本B**中的“孔”字属于**衍文**，即**多余的字**或**误增的字**。
- 在《论语》原文中，“子曰”是孔子弟子记录他言语的标准起始语，即“子”指孔子，“曰”是说话的意思，二者组合后“子曰”已经是完整的引语格式，无需在“子”前再加“孔”字。
- “孔子曰”作为典籍引语的格式，虽然在某些版本中存在，但通常是在“子曰”之后加“孔子”以明确说话者身份，而非在“子”前加“孔”。因此，此处是典型的**前增字**或**衍文**。

**文献依据**：
- 《论语》的早期文献如《汉书·艺文志》所载“鲁论语”、“齐论语”、“古文论语”等版本中，均使用“子曰”作引语形式。
- 至于“孔子曰”，在《论语》中并不常见，多用于后世的注疏或解读文本中，而非原文的引语格式。
- 因此，“孔子曰”在版本B中为**误增**，属于**衍文**。

#### **2. 位置11：「说」 vs 「悦」（替换）**

- **版本A**：第11字是“说”。
- **版本B**：将“说”替换为“悦”。

**性质判断**：
- **“说”** 是“悦”的**通假字**。在古汉语中，“说”本义为“解说、谈论”，也作“悦”的通假字，表示“喜悦、愉快”。
- “悦”字在《论语》中有明确使用，例如“有朋自远方来，不亦乐乎？”也可视为“悦乎”的通假，但由于“学而时习之，不亦悦乎？”在后世流传中**较少见**，反而出现在较晚的版本中。
- 从文义角度来看，“悦”更贴合原文语境，表达学习之后感到愉快的含义；而

In [11]:
#@title ## 3.2 AI分析《史记》异文 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，让AI分析《史记》的标点差异。
#@markdown ---

print("=" * 60)
print(" AI校勘分析：《史记·项羽本纪》")
print("=" * 60)

text_a = "项籍少时，学书不成，去；学剑，又不成。"
text_b = "项籍少时，学书不成，去学剑，又不成。"

print("\n 正在调用AI进行分析...")
print("-" * 60)

differences = find_differences(text_a, text_b)
analysis = ai_analyze(text_a, text_b, differences)

print("\n AI分析结果：")
print("-" * 60)
print(analysis)
print("\n" + "=" * 60)

 AI校勘分析：《史记·项羽本纪》

 正在调用AI进行分析...
------------------------------------------------------------

 AI分析结果：
------------------------------------------------------------
从你提供的两个古文版本来看，它们的差异在于标点符号的不同。我们来逐一分析这些差异，并评估哪个版本更接近原文。

---

### 一、逐一分析每处差异的性质

**差异位置：** 位于“学书不成，去；学剑，又不成。”与“学书不成，去学剑，又不成”之间。

**差异内容：**  
- 版本A 在“去”后使用了分号（；）  
- 版本B 则省略了分号，写作“去学剑，又不成”

#### 差异分析：

1. **标点符号的使用差异**  
   古文本中通常不使用现代的标点符号，但后来在传抄过程中出现的“句读”（即断句）会对文本的结构造成影响。严格来说，在古籍校勘中，版本A的「；」是后人加的标点，属于“**衍文**”（即传抄过程中添加的多余符号）；而版本B省略了标号，属于“**删减**”或者“**脱文**”（若认为原本应有标点，而版本B遗漏了它）。

   但实际上，在古籍中，我们并不知道原始文本是否带有分号，因此这里的「；」更可能是**后人根据语义或段落结构进行的断句所加的标点**，属于“**句读衍文**”，而非原文中必然存在。

2. **句法结构差异**  
   - 版本A：“学书不成，去；学剑，又不成。”  
     分号可能被用来分隔两个并列的事件：“学书不成，去”；“学剑，又不成”。这种断句方式意在突出“去学剑”的动作，之后再度失败。
   - 版本B：“学书不成，去学剑，又不成。”  
     无分号，三个分句按逗号连接，结构更紧凑，强调“学剑”之后也“不成”。

   这种结构差异**不涉及文本的实质性内容变化**，仅仅是**断句处理的不同**。因此，差异的性质可以归为“**标点差异**”，并不影响句子的语义。

---

### 二、判断哪个版本更接近原文

从校勘学的角度来看，古籍在流传过程中常出现**断句不当、标点误加、删减或误增**的情况。原始文本并无标点，后人根据理解加入句读。

#### 版本B 更可能更接近原文

---
<a id='part4'></a>
# 第四部分: 交互式探索实验

现在轮到你来探索了！输入自己收集的古文版本，看看AI如何分析。

In [16]:
#@title ## 4.1 交互式校勘工具 { display-mode: "form" }
#@markdown ---
#@markdown **运行此单元格**，启动交互式校勘工具。
#@markdown
#@markdown 你可以：
#@markdown - 选择预置案例快速体验
#@markdown - 输入自己收集的文本进行分析
#@markdown ---

# 预置案例
EXAMPLES = {
    "请选择案例...": ("", ""),
    "《论语》说/悦之辨": (
        "子曰：学而时习之，不亦说乎？",
        "孔子曰：学而时习之，不亦悦乎？"
    ),
    "《史记》项羽本纪": (
        "项籍少时，学书不成，去；学剑，又不成。",
        "项籍少时，学书不成，去学剑，又不成。"
    ),
    "《孟子》鱼我所欲": (
        "鱼，我所欲也；熊掌，亦我所欲也。二者不可得兼，舍鱼而取熊掌者也。",
        "鱼，我所欲也；熊掌，亦我所欲也。二者不可兼得，舍鱼而取熊掌者也。"
    ),
    "《三字经》开篇": (
        "人之初，性本善。性相近，习相远。",
        "人之初，性本善。性相近，习相远。苟不教，性乃迁。"
    ),
    "《道德经》开篇": (
        "道可道，非常道；名可名，非常名。",
        "道可道，非恒道；名可名，非恒名。"
    ),
}

# 创建控件
example_dropdown = widgets.Dropdown(
    options=list(EXAMPLES.keys()),
    value="请选择案例...",
    description='预置案例:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

text_a_input = widgets.Textarea(
    value='',
    placeholder='在此输入版本A的文本...',
    description='版本A:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='600px', height='80px')
)

text_b_input = widgets.Textarea(
    value='',
    placeholder='在此输入版本B的文本...',
    description='版本B:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='600px', height='80px')
)

analyze_button = widgets.Button(
    description='开始分析',
    button_style='primary',
    layout=widgets.Layout(width='150px', height='40px')
)

output_area = widgets.Output()

def on_example_change(change):
    """加载预置案例"""
    if change['new'] in EXAMPLES:
        text_a, text_b = EXAMPLES[change['new']]
        text_a_input.value = text_a
        text_b_input.value = text_b

def on_analyze_click(b):
    """执行分析"""
    with output_area:
        clear_output()

        text_a = text_a_input.value.strip()
        text_b = text_b_input.value.strip()

        if not text_a or not text_b:
            print(" 请输入两个版本的文本")
            return

        print("=" * 60)
        print(" 古籍校勘分析报告")
        print("=" * 60)

        # 差异检测
        differences = find_differences(text_a, text_b)
        highlighted_a, highlighted_b = highlight_differences(text_a, text_b)

        print("\n[1] 文本输入")
        print("-" * 60)
        print(f" 版本A: {text_a}")
        print(f" 版本B: {text_b}")

        print("\n[2] 差异高亮")
        print("-" * 60)
        print(f" 版本A: {highlighted_a}")
        print(f" 版本B: {highlighted_b}")

        if differences:
            print_diff_table(differences)
            print(f"\n 共检测到 {len(differences)} 处差异")

            print("\n[3] AI校勘分析")
            print("-" * 60)
            print(" 正在调用AI进行分析...\n")

            try:
                analysis = ai_analyze(text_a, text_b, differences)
                print(analysis)
            except Exception as e:
                print(f" 分析失败: {e}")
        else:
            print("\n 两个版本完全相同，无差异。")

        print("\n" + "=" * 60)

# 绑定事件
example_dropdown.observe(on_example_change, names='value')
analyze_button.on_click(on_analyze_click)

# 显示界面
print("=" * 60)
print(" 交互式古籍校勘工具")
print("=" * 60)
print("\n 使用方法：")
print(" 1. 从下拉菜单选择预置案例，或直接输入文本")
print(" 2. 点击「开始分析」按钮")
print(" 3. 查看差异检测和AI分析结果\n")

display(example_dropdown)
display(text_a_input)
display(text_b_input)
display(analyze_button)
display(output_area)

 交互式古籍校勘工具

 使用方法：
 1. 从下拉菜单选择预置案例，或直接输入文本
 2. 点击「开始分析」按钮
 3. 查看差异检测和AI分析结果



Dropdown(description='预置案例:', layout=Layout(width='400px'), options=('请选择案例...', '《论语》说/悦之辨', '《史记》项羽本纪', '《孟子…

Textarea(value='', description='版本A:', layout=Layout(height='80px', width='600px'), placeholder='在此输入版本A的文本...…

Textarea(value='', description='版本B:', layout=Layout(height='80px', width='600px'), placeholder='在此输入版本B的文本...…

Button(button_style='primary', description='开始分析', layout=Layout(height='40px', width='150px'), style=ButtonSt…

Output()

---
<a id='part5'></a>
# 第五部分: 批判性思考与讨论

AI在古籍校勘中有其价值，但也有明显的局限性。本部分引导你进行批判性思考。

---
## 5.1 AI校勘的价值与局限

### 优势

1. 快速处理大量文本，提高效率
2. 能够识别常见的通假字和异体字
3. 整合多种知识进行综合判断
4. 为初学者提供学习参考
5. 辅助发现可能遗漏的异文

### 局限

1. 缺乏对实物版本的直接考察能力
2. 可能产生看似合理但实际错误的判断
3. 对罕见异文和特殊语境理解不足
4. 无法替代专家的文献学训练和经验
5. 训练数据的偏差可能影响判断

### 建议

- AI应作为辅助工具，而非最终裁判
- 校勘结论仍需与权威校勘本对照验证
- 培养批判性思维，学会质疑AI的判断

## 5.2 思考与讨论题

1. 在你测试的案例中，AI的判断是否与专家校勘本一致？如有不同，分析原因。

2. AI在分析通假字时表现如何？它是否正确识别了通假关系？

3. 如果AI给出了错误的校勘建议，你认为原因可能是什么？

4. 你认为AI在校勘学研究中应该扮演什么角色？

5. 如何设计更好的提示词（prompt），让AI给出更准确的校勘分析？

6. 除了文字校勘，AI还能在哪些古籍研究领域发挥作用？

---

**提示**：可以将你的思考记录下来，与同学讨论或提交给老师。

---
<a id='appendix'></a>
# 附录: 预置案例库与拓展资源

本部分提供更多案例和学习资源。

## A.1 更多案例

以下案例可复制到交互式工具中进行分析：

---

### [1] 《道德经》恒/常之辨

- **版本A**: 道可道，非常道；名可名，非常名。
- **版本B**: 道可道，非恒道；名可名，非恒名。
- **说明**: 马王堆帛书本用「恒」，传世本避汉文帝刘恒讳改为「常」

---

### [2] 《诗经》关雎

- **版本A**: 关关雎鸠，在河之洲。窈窕淑女，君子好逑。
- **版本B**: 关关雎鸠，在河之州。窈窕淑女，君子好仇。
- **说明**: 洲/州、逑/仇的异文

---

### [3] 《庄子》逍遥游

- **版本A**: 北冥有鱼，其名为鲲。
- **版本B**: 北溟有鱼，其名曰鲲。
- **说明**: 冥/溟、为/曰的异文

## A.2 学习资源

### 在线古籍数据库

| 名称 | 网址 |
|------|------|
| 中国哲学书电子化计划 | https://ctext.org/zh/ |
| 国学网 | http://www.guoxue.com/ |
| 中华经典古籍库 | https://publish.ancientbooks.cn/ |

### 校勘学入门书籍

- 陈垣《校勘学释例》
- 倪其心《校勘学大纲》
- 程千帆《校雠广义》

### 进阶学习

- 尝试使用不同的大模型进行比较
- 学习Python文本处理的更多技巧
- 阅读数字人文相关论文和项目

---

**实验结束，感谢参与！**